In [1]:
!pip install python-terrier --quiet
!pip install nltk --quiet
!pip install flair --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  P

In [2]:
import pandas as pd
import numpy as np
import nltk

In [3]:
import pyterrier as pt
if not pt.started():
  pt.init()

<ipython-input-3-c43b832924c1>:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
<ipython-input-3-c43b832924c1>:3: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [10]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# **1. Data Collection**

In [4]:
import requests
from bs4 import BeautifulSoup
import csv

def extract_text_from_website(url, docno):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        title = soup.title.string if soup.title else "No Title"

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text_content = ' '.join(chunk for chunk in chunks if chunk)

        if len(text_content) < 100:
            print("Document too short:", docno)
            return None, docno, None

        print(f"Scraping successful: {title} doc id: {docno}")
        return text_content, docno, title
    else:
        print("Failed to fetch content from URL:", url)
        return None, docno, None


def save_text_to_csv(scraped_data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['docno', 'url', 'title', 'text']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in scraped_data:
            writer.writerow({'docno': data['docno'], 'url': data['url'], 'title': data['title'], 'text': data['text']})

In [5]:
URLs = {
    # Technology
    0: "https://www.simplilearn.com/importance-of-technology-article",  # importance of tech
    1: "https://medium.com/@niyajohn9495/short-article-about-technology-bebdea8736e1",
    2: "https://www.brookings.edu/articles/how-artificial-intelligence-is-transforming-the-world/",
    4: "https://www.zdnet.com/article/what-is-ai-heres-everything-you-need-to-know-about-artificial-intelligence/",
    5: "https://www.simplilearn.com/top-technology-trends-and-jobs-article",
    6: "https://scitechdaily.com/microrobots-swarm-the-seas-capturing-microplastics-and-bacteria-video/",
    7: "https://www.popsci.com/technology/best-of-whats-new-2022/",
    # Sports
    8: "https://theathletic.com/5479782/2024/05/09/football-50-most-valuable-clubs-premier-league-mls/",    # premier league
    9: "https://www.britannica.com/sports/football-soccer",                                                 # football
    10: "https://en.wikipedia.org/wiki/Tennis",                                                             # tennis
    11: "https://www.sportperformanceanalysis.com/article/what-is-performance-analysis-in-sport",
    12: "https://apnews.com/article/blazers-kings-score-bdf0f565cc2af880969cde614877c18e",
    # Zewail City
    13: "https://www.zewailcity.edu.eg/",
    14: "https://www.zewailcity.edu.eg/main/content.php?lang=en&alias=about_zewail_city",
    15: "https://www.edarabia.com/zewail-city-science-technology-6th-of-october-egypt/",
    16: "https://www.universitiesegypt.com/zewail-city-science-technology/about",
    # Covid 19
    17: "https://www.ncbi.nlm.nih.gov/books/NBK554776/",
    18: "https://www.who.int/health-topics/coronavirus#tab=tab_1",
    19: "https://www.who.int/news/item/26-04-2024-statement-on-the-antigen-composition-of-covid-19-vaccines",
    20: "https://en.wikipedia.org/wiki/COVID-19",
    21: "https://www.cdc.gov/coronavirus/2019-ncov/vaccines/index.html",
    22: "https://www.nature.com/articles/d41586-020-00502-w",
    # # Ancient places
    # 23: "https://en.wikipedia.org/wiki/Great_Pyramid_of_Giza",
    # 24: "https://www.nationalgeographic.com/history/article/giza-pyramids",
    # # 25: "https://historylists.org/architecture/top-10-historical-monuments-from-the-ancient-world.html",
    # 26: "https://www.nationalgeographic.com/expeditions/get-inspired/top-ten/best-10-ancient-ruins-archaeological-sites/",
    # 27: "https://www.smithsonianmag.com/smithsonian-institution/twelve-ancient-and-enduring-places-around-world-180976362/"
}

In [6]:
scraped_data = []

for doc_id, url in URLs.items():
    text_content, doc_id, title = extract_text_from_website(url, doc_id)
    if text_content:
        scraped_data.append({'docno': doc_id, 'url': url, 'title': title, 'text': text_content})

save_text_to_csv(scraped_data, "corpus.csv")

Scraping successful: The Importance of Technology: #5 is a Must-Know! doc id: 0
Scraping successful: Short article about Technology. Technology affects people all over the… | by Niya John | Medium doc id: 1
Scraping successful: How artificial intelligence is transforming the world doc id: 2
Scraping successful: What is AI? Everything to know about artificial intelligence | ZDNET doc id: 4
Scraping successful: 25 New Technology Trends for 2025 | Emerging Technologies 2025 doc id: 5
Failed to fetch content from URL: https://scitechdaily.com/microrobots-swarm-the-seas-capturing-microplastics-and-bacteria-video/
Scraping successful: The top 100 new technology innovations of 2022 | Popular Science doc id: 7
Scraping successful: What a list of football’s 50 most valuable clubs tells us about the global state of the sport - The Athletic doc id: 8
Scraping successful: Football (soccer) | History, Game, Rules, & Significant Players | Britannica doc id: 9
Scraping successful: Tennis - Wikipedia 

# **2. Data Preprocessing**

In [7]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
stemmer = PorterStemmer()

def clean(text):
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"@[\w]*", " ", text)
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    return text

def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = clean(text)
    # Tokenization
    tokens = word_tokenize(text)
    # Lowercasing + stopwords removing
    filtered_tokens = [token.lower() for token in tokens if token.lower() not in stop_words]
    # Stemming
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return ' '.join(stemmed_tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
corpus = pd.read_csv("/content/corpus.csv")
corpus.head()

,docno,url,title,text
0,0,https://www.simplilearn.com/importance-of-tech...,The Importance of Technology: #5 is a Must-Know!,The Importance of Technology: #5 is a Must-Kno...
1,1,https://medium.com/@niyajohn9495/short-article...,Short article about Technology. Technology aff...,Short article about Technology. Technology aff...
2,2,https://www.brookings.edu/articles/how-artific...,How artificial intelligence is transforming th...,How artificial intelligence is transforming th...
3,4,https://www.zdnet.com/article/what-is-ai-heres...,What is AI? Everything to know about artificia...,What is AI? Everything to know about artificia...
4,5,https://www.simplilearn.com/top-technology-tre...,25 New Technology Trends for 2025 | Emerging T...,25 New Technology Trends for 2025 | Emerging T...


In [11]:
corpus['processed'] = corpus['text'].apply(preprocess_text)
corpus.head()

,docno,url,title,text,processed
0,0,https://www.simplilearn.com/importance-of-tech...,The Importance of Technology: #5 is a Must-Know!,The Importance of Technology: #5 is a Must-Kno...,import technolog 5 must-know ! coursesal cours...
1,1,https://medium.com/@niyajohn9495/short-article...,Short article about Technology. Technology aff...,Short article about Technology. Technology aff...,short articl technolog technolog affect peopl ...
2,2,https://www.brookings.edu/articles/how-artific...,How artificial intelligence is transforming th...,How artificial intelligence is transforming th...,artifici intellig transform world expert event...
3,4,https://www.zdnet.com/article/what-is-ai-heres...,What is AI? Everything to know about artificia...,What is AI? Everything to know about artificia...,ai everyth know artifici intellig zdnet x tren...
4,5,https://www.simplilearn.com/top-technology-tre...,25 New Technology Trends for 2025 | Emerging T...,25 New Technology Trends for 2025 | Emerging T...,25 new technolog trend 2025 emerg technolog 20...


# **3. Indexing**

In [12]:
inverted_index = {}

for index, row in corpus.iterrows():
    doc_id = row["docno"]
    text = row["processed"]

    words = text.split()

    for word in words:
        if word not in inverted_index:
            inverted_index[word] = set()

        inverted_index[word].add(doc_id)

for word, doc_ids in inverted_index.items():
        print(f"'{word}': {list(doc_ids)}")

'import': [0, 2, 4, 5, 7, 10, 11, 18, 19, 20, 22]
'technolog': [0, 1, 2, 4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20]
'5': [0, 1, 2, 4, 5, 7, 8, 9, 10, 12, 15, 16, 18, 19, 20]
'must-know': [0]
'!': [0, 4, 5, 7, 9, 10, 12, 15, 16, 22]
'coursesal': [0, 5]
'coursesfor': [0, 5]
'businessresourcesfre': [0, 5]
'coursesfre': [0, 5]
'resourcesmorereviewsbecom': [0, 5]
'partnerbecom': [0, 5]
'instructor': [0, 2, 5]
'loginsoftwar': [0]
'developmentdata': [0]
'scienc': [0, 2, 4, 5, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 20, 22]
'&': [0, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 22]
'busi': [0, 2, 4, 5, 8, 12, 15]
'analyticsai': [0, 5]
'machin': [0, 2, 4, 5, 7, 10, 20]
'learningproject': [0, 5]
'managementcyb': [0, 5]
'securitycloud': [0, 5]
'computingdevopsbusi': [0, 5]
'leadershipqu': [0, 5]
'managementsoftwar': [0, 5]
'developmentagil': [0, 5]
'scrumit': [0, 5]
'servic': [0, 2, 4, 5, 7, 10, 12, 20, 22]
'architecturedigit': [0, 5]
'marketingbig': [0, 5]
'datacar': [0, 5]
'fast-track

In [13]:
term_frequencies = {}

for index, row in corpus.iterrows():
    doc_id = row["docno"]
    text = row["processed"]

    words = text.split()

    for word in words:
        if word not in term_frequencies:
            term_frequencies[word] = {}

        if doc_id not in term_frequencies[word]:
            term_frequencies[word][doc_id] = 0

        term_frequencies[word][doc_id] += 1

print(term_frequencies)

{'import': {0: 14, 2: 12, 4: 2, 5: 2, 7: 4, 10: 2, 11: 1, 18: 2, 19: 4, 20: 5, 22: 2}, 'technolog': {0: 86, 1: 19, 2: 54, 4: 13, 5: 73, 7: 18, 9: 1, 10: 6, 11: 15, 12: 11, 13: 2, 14: 2, 15: 16, 16: 40, 18: 2, 20: 3}, '5': {0: 2, 1: 1, 2: 4, 4: 6, 5: 3, 7: 10, 8: 2, 9: 3, 10: 21, 12: 6, 15: 3, 16: 4, 18: 5, 19: 20, 20: 97}, 'must-know': {0: 1}, '!': {0: 5, 4: 1, 5: 12, 7: 1, 9: 1, 10: 4, 12: 4, 15: 1, 16: 4, 22: 1}, 'coursesal': {0: 1, 5: 1}, 'coursesfor': {0: 1, 5: 1}, 'businessresourcesfre': {0: 1, 5: 1}, 'coursesfre': {0: 1, 5: 1}, 'resourcesmorereviewsbecom': {0: 1, 5: 1}, 'partnerbecom': {0: 1, 5: 1}, 'instructor': {0: 1, 2: 1, 5: 1}, 'loginsoftwar': {0: 1}, 'developmentdata': {0: 1}, 'scienc': {0: 4, 2: 7, 4: 2, 5: 7, 7: 16, 8: 1, 9: 4, 11: 2, 12: 6, 13: 2, 14: 2, 15: 16, 16: 40, 18: 3, 20: 59, 22: 6}, '&': {0: 6, 2: 52, 4: 6, 5: 15, 7: 4, 8: 3, 9: 33, 10: 8, 11: 4, 12: 8, 13: 10, 14: 10, 15: 8, 16: 1, 18: 3, 20: 15, 22: 10}, 'busi': {0: 14, 2: 14, 4: 17, 5: 24, 8: 2, 12: 13, 15: 

In [14]:
corpus['docno'] = corpus['docno'].astype(str)
indexer = pt.DFIndexer("./myFirstIndex", overwrite=True)

index_ref = indexer.index(corpus["processed"], corpus["docno"])

index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

<ipython-input-14-87dda3a6aef4>:2: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer("./myFirstIndex", overwrite=True)


Number of documents: 19
Number of terms: 7653
Number of postings: 16466
Number of fields: 0
Number of tokens: 54009
Field names: []
Positions:   false



# **4. Query Processing**

In [15]:
# retieval model
tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)

<ipython-input-15-8ce8492b68d6>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"},num_results=10)


In [16]:
query = 'sports'
query = preprocess_text(query)
results = tfidf_retr.search(query)
results

,qid,docid,docno,rank,score,query
0,1,9,11,0,2.227873,sport
1,1,7,9,1,2.146672,sport
2,1,8,10,2,2.068764,sport
3,1,6,8,3,2.056757,sport
4,1,10,12,4,2.013040,sport
5,1,5,7,5,1.823364,sport
6,1,13,15,6,1.504523,sport


# **5. Query expansion**

In [17]:
!pip install transformers --quiet

In [18]:
from transformers import AutoTokenizer, AutoModel
import tensorflow_hub as hub
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
def preprocess_document(doc):
    tokens = word_tokenize(doc)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

def retrieve_top_documents(query, num_results = 3):
    tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=num_results)
    query = preprocess_text(query)
    results = tfidf_retr.search(query)
    return results

def get_top_document_texts(results, corpus_df):
    top_docnos = results['docno'].tolist()
    top_documents_df = corpus_df[corpus_df['docno'].isin(top_docnos)]
    return top_documents_df['text'].tolist()

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
elmo = hub.load("https://tfhub.dev/google/elmo/3")

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def process_text(text):
    # Remove special characters
    text = re.sub(r"[^\w\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    return text

def expand_with_elmo(original_query, feedback_docs, num_expansion_terms=10):
    original_query_terms = process_text(original_query).split()
    original_query_embeddings = elmo.signatures["default"](tf.constant([" ".join(original_query_terms)]))["elmo"][0]

    feedback_terms = []
    for doc_text in feedback_docs:
        feedback_terms.extend(process_text(doc_text).split())
    feedback_embeddings = elmo.signatures["default"](tf.constant([" ".join(feedback_terms)]))["elmo"][0]

    similar_terms = []
    for i, term_embedding in enumerate(feedback_embeddings):
        term_embedding_2d = term_embedding.numpy().reshape(1, -1)
        similarity_scores = [
            cosine_similarity(term_embedding_2d, original_query_embedding.numpy().reshape(1, -1))[0][0]
            for original_query_embedding in original_query_embeddings
        ]
        most_similar_index = np.argmax(similarity_scores)
        similar_terms.append(feedback_terms[i])

    processed_similar_terms = [process_text(term) for term in similar_terms if process_text(term) not in stop_words]

    expansion_terms = processed_similar_terms[:num_expansion_terms]

    expanded_query = original_query + " " + " ".join(expansion_terms)

    return expanded_query

In [22]:
query = "technology"
results = retrieve_top_documents(query)
top_document_texts = get_top_document_texts(results, corpus)
expanded_query = expand_with_elmo(query, top_document_texts)
print("Original Query:", query)
print("Expanded Query:", expanded_query)

<ipython-input-19-ad74e4896f78>:7: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=num_results)


Original Query: technology
Expanded Query: technology importance technology 5 mustknow coursesall coursesfor businessresourcesfree coursesfree resourcesmorereviewsbecome partnerbecome


# **6. User Interface**

In [23]:
!pip install pyngrok --quiet

In [ ]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import socket
from IPython.display import display, HTML
import ipywidgets as widgets
from nltk.tokenize import word_tokenize


corpus['docno'] = corpus['docno'].astype(str)
indexer = pt.DFIndexer("./myFirstIndex", overwrite=True)
index_ref = indexer.index(corpus["processed"], corpus["docno"], corpus["url"], corpus["text"])
index = pt.IndexFactory.of(index_ref)
tokens2 =  corpus['processed'].apply(word_tokenize)

app = Flask(__name__)

ngrok.set_auth_token('Replace with your ngrok token')
public_url = ngrok.connect(5000)
print('NGROK Tunnel URL:', public_url)


app = Flask(__name__)


HTML_TEMPLATE = '''
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>Search Engine</title>
  <style>
    body {
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      margin: 0;
      padding: 0;
      text-align: center;
      background-color: #e8eff1;
      color: #333;
    }
    .search-box {
      background-color: #2c3e50;
      padding: 40px 0;
      border-bottom: 5px solid #34495e;
    }
    .search-form {
      display: flex;
      justify-content: center;
      align-items: center;
      margin-bottom: 20px;
    }
    .search-input {
      width: 50%;
      height: 50px;
      padding: 0 24px;
      font-size: 18px;
      border: 2px solid #34495e;
      border-radius: 25px;
      margin-right: 10px;
      box-shadow: 0 4px 6px rgba(50, 50, 93, 0.11), 0 1px 3px rgba(0, 0, 0, 0.08);
      transition: all 0.15s ease;
    }
    .search-input:focus {
      outline: none;
      border-color: #3498db;
      box-shadow: 0 4px 6px rgba(52, 152, 219, 0.4), 0 1px 3px rgba(0, 0, 0, 0.08);
    }
    .search-button {
      height: 50px;
      width: 150px;
      background-color: #3498db;
      color: white;
      font-size: 18px;
      border: none;
      border-radius: 25px;
      cursor: pointer;
      box-shadow: 0 4px 6px rgba(50, 50, 93, 0.11), 0 1px 3px rgba(0, 0, 0, 0.08);
      transition: all 0.15s ease;
    }
    .search-button:hover {
      background-color: #2980b9;
      box-shadow: 0 7px 14px rgba(50, 50, 93, 0.1), 0 3px 6px rgba(0, 0, 0, 0.08);
    }
    .results-container {
      margin-top: 30px;
      padding: 20px;
      text-align: left;
      background-color: #ffffff;
      border-radius: 8px;
      box-shadow: 0 4px 8px rgba(0,0,0,0.1);
      width: 80%;
      max-width: 800px;
      margin-left: auto;
      margin-right: auto;
    }
    .result-item {
      background-color: #f8f9fa;
      margin-bottom: 15px;
      padding: 15px;
      border: 1px solid #e0e0e0;
      border-radius: 4px;
      transition: transform 0.2s ease, box-shadow 0.2s ease;
    }
    .result-item:hover {
      transform: translateY(-3px);
      box-shadow: 0 6px 12px rgba(0,0,0,0.15);
    }
    .result-link {
      text-decoration: none;
      color: #3498db;
      font-size: 20px;
      font-weight: bold;
      display: block;
      margin: -15px;
      padding: 15px;
    }
    .result-link:hover {
      text-decoration: underline;
    }
    .no-results {
      color: #95a5a6;
      font-size: 18px;
      text-align: center;
    }
  </style>
</head>
<body>
  <div class="search-box">
    <form class="search-form" action="/search" method="post"> 
      <input class="search-input" type="text" name="query" placeholder="Enter search terms..." value="{{ query }}">
      <input class="search-button" type="submit" value="Search">
    </form>
  </div>
  <div class="results-container">
    {% if query %}
      <h2 style="color: #333;">Results for: "{{ query }}"</h2>
      {% if results %}
        {% for title, url in results %}
          <div class="result-item">
            <a class="result-link" href="{{ url }}">{{ title }}</a>
          </div>
        {% endfor %}
      {% else %}
        <p class="no-results">No results found.</p>
      {% endif %}
    {% endif %}
  </div>
</body>
</html>

'''

@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route("/search", methods=["POST"])
def search():
    query = request.form['query']
    search_results = []
    original_query = query

    tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)
    query = preprocess_text(query)
    tfidf_results = tfidf_retr.search(query)

    # Handle results
    for idx, row in tfidf_results.iterrows():
        docno = row['docno']
        link = corpus[corpus['docno'] == docno]['url'].values[0]
        title = corpus[corpus['docno'] == docno]['title'].values[0]
        search_results.append((title, link))

    return render_template_string(HTML_TEMPLATE, query = original_query, results = search_results)

if __name__ == "__main__":
    app.run(port=5000)

<ipython-input-25-463f2b1a86cc>:10: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  indexer = pt.DFIndexer("./myFirstIndex", overwrite=True)


NGROK Tunnel URL: NgrokTunnel: "https://5986-34-57-59-139.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 23:36:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 23:36:48] "GET /favicon.ico HTTP/1.1" 404 -
<ipython-input-25-463f2b1a86cc>:162: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 23:37:01] "POST /search HTTP/1.1" 200 -
<ipython-input-25-463f2b1a86cc>:162: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)
INFO:werkzeug:127.0.0.1 - - [23/Apr/2025 23:

In [26]:
ngrok.kill()

# **7. Evaluation**

In [27]:
def evaluate_accuracy(df, ground_truth):
    results = {}
    for query, group in df.groupby('query'):
        retrieved_docs = group['docno'].astype(str).tolist()  # Convert docno to string for comparison

        relevant_docs = ground_truth.get(query, [])

        if relevant_docs:  # Check if relevant documents exist for the query
            precision = len(set(retrieved_docs) & set(relevant_docs)) / len(retrieved_docs)
            recall = len(set(retrieved_docs) & set(relevant_docs)) / len(relevant_docs)
            f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
            results[query] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}
        else:
            print(f"No ground truth relevant documents found for query: {query}")

    return results


test_queries = {
    "technology": [0, 1, 2, 4, 5, 6],
    "sports": [7, 8, 9, 10, 11],
    "zewail city": [12, 13, 14, 15],
    "covid 19": [16, 17, 18, 19, 20, 21]
}

ground_truth = {
    "technology": ["0", "1", "2", "4", "5", "6"],
    "sports": ["7", "8", "9", "10", "11"],
    "zewail city": ["13", "14", "15", "16"],
    "covid 19": ["17", "18", "19", "20", "21", "22"]
}

search_engine = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)

for query, docnos in test_queries.items():
    query_results = search_engine.search(query)

    accuracy_results = evaluate_accuracy(query_results, ground_truth)

    print(f"Query: {query}")
    print("Accuracy Results:")
    print(accuracy_results)
    print()

Query: technology
Accuracy Results:
{'technology': {'precision': 0.5, 'recall': 0.8333333333333334, 'f1_score': 0.625}}

Query: sports
Accuracy Results:
{'sports': {'precision': 0.7142857142857143, 'recall': 1.0, 'f1_score': 0.8333333333333333}}

Query: zewail city
Accuracy Results:
{'zewail city': {'precision': 0.4, 'recall': 1.0, 'f1_score': 0.5714285714285715}}

Query: covid 19
Accuracy Results:
{'covid 19': {'precision': 0.4, 'recall': 0.6666666666666666, 'f1_score': 0.5}}



<ipython-input-27-d7f9942a3b12>:33: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)


In [28]:
import time

def measure_retrieval_time(search_engine, queries):
    for query in queries:
        start_time = time.time()
        query_results = search_engine.search(query)
        end_time = time.time()
        retrieval_time = end_time - start_time
        print(f"Query: {query}")
        print(f"Retrieval time: {retrieval_time} seconds")
        print()

def evaluate_search_engine(search_engine, queries):
    total_time = 0
    total_results = 0

    for query in queries:
        start_time = time.time()
        query_results = search_engine.search(query)
        end_time = time.time()

        query_time = end_time - start_time
        total_time += query_time

        total_results += len(query_results)


    avg_time_per_query = total_time / len(queries)
    avg_results_per_query = total_results / len(queries)

    return avg_time_per_query, avg_results_per_query

measure_retrieval_time(search_engine, test_queries.keys())

avg_time, avg_results = evaluate_search_engine(search_engine, test_queries.keys())

print("Average retrieval time per query:", avg_time)
print("Average number of results per query:", avg_results)

Query: technology
Retrieval time: 0.012253046035766602 seconds

Query: sports
Retrieval time: 0.01418924331665039 seconds

Query: zewail city
Retrieval time: 0.02589106559753418 seconds

Query: covid 19
Retrieval time: 0.01859140396118164 seconds

Average retrieval time per query: 0.022370755672454834
Average number of results per query: 9.25
